In [1]:
import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import os
import argparse
import time
from im2scene import config
from im2scene.checkpoints import CheckpointIO
import logging
logger_py = logging.getLogger(__name__)

/home/matthewachan/environments/scatternet/lib/python3.8/site-packages/kornia/augmentation/augmentation.py:1872: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(


In [2]:
import os
import urllib
import torch
from torch.utils import model_zoo
import shutil
import datetime

In [3]:
import mrcfile
from tqdm import tqdm

In [4]:
device = torch.device('cuda:0')

In [5]:
out_dir = 'out/FFHQ_256b/'

In [10]:
cfg = config.load_config("configs/256res/FFHQ_256b.yaml", 'configs/default.yaml')

In [11]:
# Load model

In [12]:
model = config.get_model(cfg, device=device, len_dataset=1)

In [13]:
checkpoint_io = CheckpointIO(out_dir, model=model)
try:
    load_dict = checkpoint_io.load('model.pt')
    print("Loaded model checkpoint.")
except FileExistsError:
    load_dict = dict()
    print("No model checkpoint found.")


out/FFHQ_256b/model.pt
=> Loading checkpoint from local file...
Loaded model checkpoint.


In [14]:
def create_samples(N=256, voxel_origin=[0, 0, 0], cube_length=2.0):
    x, y, z = voxel_origin
    halfcube = cube_length/2
    x = torch.arange(0, N) * cube_length / (N-1) + x - halfcube
    y = torch.arange(0, N) * cube_length / (N-1) + y - halfcube
    z = torch.arange(0, N) * cube_length / (N-1) + z - halfcube

    grid = torch.stack(torch.meshgrid(x, y, z), dim=-1)
    
    samples = grid.reshape(1, -1, 3)

    return samples

In [15]:
batch_size=1
z_dim=256
torch.manual_seed(0)
shape_z = torch.randn(batch_size, z_dim).to(device)
app_z = torch.randn(batch_size, z_dim).to(device)

In [16]:
# CREATE SAMPLES
voxel_res = 256

points = create_samples(voxel_res).to(device)
raydirs = torch.zeros_like(points) # shape is independent of raydir

In [18]:
## RUN SAMPLES

In [23]:
MAX_SAMPLES_PER_BATCH = int(2**20)

# batch across samples:
with torch.no_grad():
    sigma = torch.zeros(points.shape[:2]).to(device)

    i = 0
    with tqdm(total = sigma.shape[1]) as pbar:
        while i < sigma.shape[1]:
            _, sigma[:, i:i+MAX_SAMPLES_PER_BATCH] = model.generator.decoder(points[:, i:i+MAX_SAMPLES_PER_BATCH], raydirs[:, i:i+MAX_SAMPLES_PER_BATCH], shape_z, app_z)

            i += MAX_SAMPLES_PER_BATCH
            pbar.update(MAX_SAMPLES_PER_BATCH)

    sigma = torch.sigmoid(sigma)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16777216/16777216 [00:00<00:00, 33591496.90it/s]


In [24]:
sigma = sigma.reshape(1, voxel_res, voxel_res, voxel_res).cpu().numpy()

In [25]:
with mrcfile.new_mmap(os.path.join(out_dir, f'shape.mrc'), overwrite=True, shape=sigma.shape, mrc_mode=2) as mrc:
    mrc.data[:] = sigma